In [0]:
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
PATH='/content/gdrive/My Drive/ML Project/train.csv'

In [5]:
data = pd.read_csv(PATH)
print('총 샘플의 수 :',len(data))
train_data=data.copy()
train_data['title'].nunique(), data['text'].nunique() #기사제목, 기사내용 중복 내용 확인
# 윈도우 바탕화면에서 작업한 저자의 경우에는
# data = pd.read_csv(r'C:\Users\USER\Desktop\spam.csv',encoding='latin1')
train_data.drop_duplicates(subset=['text'], inplace=True) # v2 열에서 중복인 내용이 있다면 중복 제거
print('중복 제거 후 샘플의 수 :',len(train_data))

총 샘플의 수 : 20800
중복 제거 후 샘플의 수 : 20387


In [0]:
# importing neural network libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, RNN, SpatialDropout1D
from tensorflow.keras.models import load_model

In [0]:
train_data = train_data.set_index('id', drop = True)

In [8]:
print(train_data.shape)
train_data.head()

(20387, 4)


,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
train_data.isnull().sum()

title      518
author    1854
text         1
label        0
dtype: int64

In [10]:
train_data[['title', 'author']] = train_data[['title', 'author']].fillna(value = 'Missing')
train_data = train_data.dropna()
train_data.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [11]:
length = []
[length.append(len(str(text))) for text in train_data['text']]
train_data['length'] = length
train_data.head()

,title,author,text,label,length
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938


In [12]:
train_data = train_data.drop(train_data['text'][train_data['length'] < 50].index, axis = 0)
print('기사의 최대길이 : %d' % max(j for j in train_data['length']))

기사의 최대길이 : 142961


In [0]:
max_features = 4500

In [0]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = train_data['text'])
X = tokenizer.texts_to_sequences(texts = train_data['text'])

In [15]:
X = pad_sequences(sequences = X, maxlen = max_features, padding = 'pre')
print(X.shape)

(20263, 4500)


In [16]:
print(X.shape)
y = train_data['label'].values
print(y.shape)
print(X)
m=len(X)

(20263, 4500)
(20263,)
[[   0    0    0 ... 1096    5 4201]
 [   0    0    0 ... 3733  139 1166]
 [   0    0    0 ...    3  207  671]
 ...
 [   0    0    0 ... 3741  778   11]
 [   0    0    0 ...   10   59 3805]
 [   0    0    0 ... 1565 2266 3643]]


In [17]:
n_of_train = int(len(X) * 0.8)
n_of_test = int(len(X) - n_of_train)
print('훈련데이터 수: '+ str(n_of_train))
print('검증데이터 수: '+ str(n_of_test))
# train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=1)
print(X_valid.shape)
print(y_valid.shape)
print(X_train.shape)
print(y_train.shape)

훈련데이터 수: 16210
검증데이터 수: 4053
(4053, 4500)
(4053,)
(16210, 4500)
(16210,)


In [0]:
model = Sequential()
model.add(Embedding(4500, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

In [0]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/gdrive/My Drive/ML Project/LSTMbest_model2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [19]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=30, batch_size=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/30
1297/1297 [==============================] - ETA: 0s - loss: 0.2773 - acc: 0.8915
Epoch 00001: val_acc improved from -inf to 0.92813, saving model to /content/gdrive/My Drive/ML Project/LSTMbest_model2.h5
1297/1297 [==============================] - 171s 132ms/step - loss: 0.2773 - acc: 0.8915 - val_loss: 0.2010 - val_acc: 0.9281
Epoch 2/30
1297/1297 [==============================] - ETA: 0s - loss: 0.1518 - acc: 0.9497
Epoch 00002: val_acc improved from 0.92813 to 0.93831, saving model to /content/gdrive/My Drive/ML Project/LSTMbest_model2.h5
1297/1297 [==============================] - 172s 132ms/step - loss: 0.1518 - acc: 0.9497 - val_loss: 0.1912 - val_acc: 0.9383
Epoch 3/30
1297/1297 [==============================] - ETA: 0s - loss: 0.1260 - acc: 0.9578
Epoch 00003: val_acc did not improve from 0.93831
1297/1297 [==============================] - 173s 133ms/step - loss: 0.1260 - acc: 0.9578 - val_loss: 0.1795 - val_acc: 0.9217
Epoch 4/30
1297/1297 [===================

In [18]:
loaded_model = load_model('/content/gdrive/My Drive/ML Project/LSTMbest_model2.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_valid, y_valid)[1]))

127/127 [==============================] - 23s 179ms/step - loss: 0.1243 - acc: 0.9689

 테스트 정확도: 0.9689


In [20]:
import numpy as np
from numpy import argmax
loaded_model = load_model('/content/gdrive/My Drive/ML Project/LSTMbest_model2.h5')
##xhat = X[1]
##yhat = loaded_model.predict(xhat)
##print('Predict : ' + str(yhat))

n=50

xhat_idx = np.random.choice(X_valid.shape[0], n)
xhat = X_valid[xhat_idx]
yhat = loaded_model.predict_classes(xhat)

print(X_valid.shape, y_valid.shape)

for i in range(n):
    if(yhat[i]==0):
      print('Index: ' + str(xhat_idx[i]) + ' Label : ' + str(y_valid[xhat_idx[i]]) + ' Predict : True '+ str(yhat[i]))
    else:
      print('Index: ' + str(xhat_idx[i]) + ' Label : ' + str(y_valid[xhat_idx[i]]) + ' Predict : False '+ str(yhat[i]))

    ##print('Predict : ' + str(yhat[i]))

##if(y_valid[xhat_idx[0]]==yhat[0]):
   ## print("same")  

(4053, 4500) (4053,)
Index: 458 Label : 0 Predict : True [0]
Index: 1676 Label : 0 Predict : True [0]
Index: 1271 Label : 0 Predict : True [0]
Index: 491 Label : 0 Predict : True [0]
Index: 2935 Label : 0 Predict : True [0]
Index: 1832 Label : 0 Predict : True [0]
Index: 364 Label : 1 Predict : False [1]
Index: 3638 Label : 1 Predict : False [1]
Index: 2909 Label : 1 Predict : False [1]
Index: 2753 Label : 1 Predict : False [1]
Index: 149 Label : 1 Predict : False [1]
Index: 1482 Label : 0 Predict : True [0]
Index: 1193 Label : 0 Predict : True [0]
Index: 3481 Label : 0 Predict : True [0]
Index: 1016 Label : 1 Predict : False [1]
Index: 634 Label : 1 Predict : False [1]
Index: 1320 Label : 0 Predict : True [0]
Index: 114 Label : 1 Predict : False [1]
Index: 320 Label : 0 Predict : True [0]
Index: 2509 Label : 1 Predict : False [1]
Index: 1777 Label : 0 Predict : True [0]
Index: 519 Label : 1 Predict : False [1]
Index: 1473 Label : 0 Predict : True [0]
Index: 22 Label : 0 Predict : True

In [0]:
## test.csv 사용하여 예측
PATH='/content/gdrive/My Drive/ML Project/test.csv'

In [0]:
data = pd.read_csv(PATH)
print('총 샘플의 수 :',len(data))
train_data=data.copy()
train_data['title'].nunique(), data['text'].nunique() #기사제목, 기사내용 중복 내용 확인
# 윈도우 바탕화면에서 작업한 저자의 경우에는
# data = pd.read_csv(r'C:\Users\USER\Desktop\spam.csv',encoding='latin1')
train_data.drop_duplicates(subset=['text'], inplace=True) # v2 열에서 중복인 내용이 있다면 중복 제거
print('중복 제거 후 샘플의 수 :',len(train_data))

총 샘플의 수 : 5200
중복 제거 후 샘플의 수 : 5127


In [0]:
train_data = train_data.set_index('id', drop = True)

In [0]:
print(train_data.shape)
train_data.head()

(5127, 3)


,title,author,text
id,,,
20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [0]:
train_data.isnull().sum()

title     120
author    490
text        1
dtype: int64

In [0]:
train_data[['title', 'author']] = train_data[['title', 'author']].fillna(value = 'Missing')
train_data = train_data.dropna()
train_data.isnull().sum()

title     0
author    0
text      0
dtype: int64

In [0]:
length = []
[length.append(len(str(text))) for text in train_data['text']]
train_data['length'] = length
train_data.head()

,title,author,text,length
id,,,,
20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",8015
20801,Russian warships ready to strike terrorists ne...,Missing,Russian warships ready to strike terrorists ne...,1559
20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,4547
20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",3578
20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,542


In [0]:
train_data = train_data.drop(train_data['text'][train_data['length'] < 50].index, axis = 0)
print('기사의 최대길이 : %d' % max(j for j in train_data['length']))

기사의 최대길이 : 107014


In [0]:
max_features = 4500

In [0]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = train_data['text'])
X = tokenizer.texts_to_sequences(texts = train_data['text'])

In [0]:
print(X[2000])

[2493, 3970, 3, 5, 668, 3, 908, 2406, 173, 177, 68, 300, 77, 4, 25, 20, 464, 6, 49, 3, 1, 24, 4414, 77, 112, 1239, 3, 8, 827, 2, 3330, 104, 764, 7, 16, 35, 48, 6, 571, 5, 87, 415, 714, 31, 368, 256, 56, 2313, 2, 125, 40, 14, 38, 65, 136, 67, 11, 16, 264, 958, 1100, 300, 77, 2354, 1318, 6, 3, 884, 300, 77, 2628, 24, 2287, 2235, 33, 4326, 1221, 3656, 37, 19, 733, 1, 3569, 12, 530, 507, 10, 390, 19, 1, 76, 3, 1, 3569, 1, 144, 35, 43, 1457, 83, 611, 40, 458, 4115, 13, 4, 46, 12, 1892, 7, 1, 3, 611, 82, 568, 2, 72, 90, 46, 48, 415, 15, 29, 1668, 9, 1, 2167, 5, 3599, 12, 113, 1442, 18, 29, 3060, 511, 33, 965, 7, 910, 13, 77, 1219, 48, 24, 571, 1, 415, 53, 1, 511, 1021, 46, 12, 47, 6, 1, 3970, 77, 8, 827, 10, 90, 341, 2, 392, 13, 1, 511, 4, 611, 7, 117, 1, 3, 1, 3970, 2813, 13, 1, 87, 415, 44, 388, 2, 1, 103, 144, 6, 4182, 28, 276, 74, 49, 145, 63, 77, 35, 748, 10, 1518, 1012, 13, 25, 860, 120, 34, 1, 546, 258, 233, 887, 34, 104, 910, 13, 306, 6, 25, 9, 5, 976, 3561, 1787, 1442, 105, 81, 18, 

In [0]:
X_test = pad_sequences(sequences = X, maxlen = max_features, padding = 'pre')
print(X_test.shape)

(5093, 4500)


In [0]:
import numpy as np
from numpy import argmax
loaded_model = load_model('/content/gdrive/My Drive/ML Project/best_model.h5')
##xhat = X[1]
##yhat = loaded_model.predict(xhat)
##print('Predict : ' + str(yhat))

n=50

xhat_idx = np.random.choice(X_valid.shape[0], n)
xhat = X_valid[xhat_idx]
yhat = loaded_model.predict_classes(xhat)

print(X_valid.shape, y_valid.shape)

for i in range(n):
    if(yhat[i]==0):
      print('Index: ' + str(xhat_idx[i]) + ' Label : ' + str(y_valid[xhat_idx[i]]) + ' Predict : True '+ str(yhat[i]))
    else:
      print('Index: ' + str(xhat_idx[i]) + ' Label : ' + str(y_valid[xhat_idx[i]]) + ' Predict : False '+ str(yhat[i]))

    ##print('Predict : ' + str(yhat[i]))

##if(y_valid[xhat_idx[0]]==yhat[0]):
   ## print("same")  

ID: 991 Predict : False [1]
ID: 291 Predict : True [0]
ID: 4398 Predict : False [1]
ID: 2443 Predict : False [1]
ID: 4951 Predict : True [0]
ID: 3385 Predict : False [1]
ID: 2084 Predict : False [1]
ID: 2498 Predict : False [1]
ID: 1056 Predict : True [0]
ID: 4078 Predict : True [0]
ID: 4699 Predict : True [0]
ID: 977 Predict : False [1]
ID: 3260 Predict : True [0]
ID: 393 Predict : True [0]
ID: 3318 Predict : False [1]
ID: 988 Predict : False [1]
ID: 1659 Predict : False [1]
ID: 4917 Predict : True [0]
ID: 4790 Predict : False [1]
ID: 3009 Predict : True [0]
ID: 3759 Predict : False [1]
ID: 156 Predict : False [1]
ID: 1261 Predict : False [1]
ID: 1927 Predict : False [1]
ID: 2172 Predict : False [1]
ID: 2172 Predict : False [1]
ID: 907 Predict : True [0]
ID: 2991 Predict : False [1]
ID: 825 Predict : True [0]
ID: 89 Predict : True [0]
ID: 2231 Predict : False [1]
ID: 1484 Predict : True [0]
ID: 1237 Predict : True [0]
ID: 3162 Predict : False [1]
ID: 1383 Predict : True [0]
ID: 766 Pr